<center><img src=img/MScAI_brand.png width=80%></center>

# The Substitution Model and the Environment Model

### The substitution model

Question: how does Python interpret a complex expression?

*From the inside out*.

An *expression* is a part of a program, consisting of values, variables, operators, and function calls. It can be simple, like `3`, or complex, like `math.sin(math.sqrt(x+12))`. Anyway, it has a single *value* (which could be a compound data structure -- but don't confuse a complex expression with a compound data structure). How can we tell what that value is?



You are already familiar with the inside-out *substitution model* from basic arithmetic. To evaluate $\sin(\sqrt{x+12})$: first get the value of $x$, then $x+12$, then $\sqrt{x+12}$, etc. At each step, we mentally substitute in the value (e.g. $16$, if we suppose $x=4$) in place of the sub-expression (e.g. $x+12$).

`math.sin(math.sqrt(x+12))`

= `math.sin(math.sqrt(16))` (because $x=4$)

= `sin(4.0)` (because $\sqrt{16}=4$, and `sqrt` always returns a floating-point number)

= `-.7568` (because $\sin(4) = -.7568\ldots$)

Thus, the *expression* `math.sin(math.sqrt(x+12))` has the *value* `-.7568...`.

The substitution model in programming is just an extension of the substitution model in basic arithmetic. The value doesn't have to be a floating-point number as here. 


For example: suppose we have two functions, $f$ and $g$, and we want $f(g(x))$ for some $x$. Just like in maths, we can then write:

```
z = f(g(x))
```

If it helps, we can think of this as:

```
y = g(x)
z = f(y)
```

Here's an example using the same concept, but with list access instead of function calls.

In [ ]:
M = [
    [0, 1, 2, 3],
    [4, 5, 6, 7],
    [8, 9, 10, 11]
]

We can access elements as follows:

```python
print(M[1][2])
```

What does that expression mean? *Apply the substitution model*:

```python
print(M[1][2])
```
is the same as 
```python
T = M[1]     # == [4, 5, 6, 7]
print(T[2])  # == 6
```

How did I know to take `M[1]` before `[1][2]`?

That is, why is it treated as `(M[1])[2])` as opposed to `M([1][2])`?

This is one of the rules of operator precedence in the language. The `[]` operator binds left-to-right. It has to be this way since `[1][2]` doesn't mean anything on its own!

### The substitution model
To evaluate a complex expression:

* Find the *innermost* simple expression, evaluate it, and *substitute it* with the result.
* Repeat until the result is simple.


### Namespaces, Scope, and the Environment Model

When the innermost simple expression is a *function call*, the story is the same (evaluate it and substitute the result), but we also have to think about variable *scope*.

### Namespaces

In Python, a *namespace* is an environment in which names ("variables") are mapped to values. For example, within a function:
```python
def f(x):
    y = 3
    return x + y
f(2)
```
the name `y` is mapped to an object of type `int` with value `3`. The name `x` is mapped to another object.

"Mapped to" means that when Python is interpreting the code and encounters a name `y`, it can *look it up in the namespace* and find the corresponding value.

So, it's useful to think of a namespace as being a bit like a dictionary:
```python
f_namespace = {"x": 2, "y": 3}
```

Every program begins with the *global* namespace. When we call a function, a new *local* namespace is created, nested inside the current namespace.


Objects, functions, and modules all create their own namespaces when they are called/created/imported. E.g. a name can be created inside a function called by a function attached to an object created by a function attached to an object created by a module. 

For example, in a file called `math.py` there is an expression `pi = 3.14159...`. (Ok, we are simplifying a bit here.)

`pi` is in the `math` namespace, and becomes accessible as `math.pi`. 

### Scope

Names are accessible (*in scope*) from outer to inner ("nested") frames, but not the other way around.

In [1]:
z = 4
def f(x):
    y = 3
    return x + y + z # z is in scope!
f(2)

9

In [2]:
z = 4
def f(x):
    y = 3
    return x + y + z # z is in scope!
f(2)
print(y) # y is NOT in scope

NameError: name 'y' is not defined

Thus, namespaces can "hide" names from each other.

This is essential to creating large programs and re-using code by other people. In particular, it means that I can use `x` as a variable name without fearing a clash with a variable in some function in the `math` library, or in some module my co-worker is writing.

**Further reading**: https://sebastianraschka.com/Articles/2014_python_scope_and_namespaces.html

### How to call a function
* Evaluate the arguments passed-in;
* Create a new namespace with new variables (names given by formal parameters, values given by above evaluation);
* Run the function body;
* If it refers to any names not among the new variables, look in the enclosing namespace (recursively if necessary);
* Return a value and substitute it for the function call in the enclosing namespace;
* Delete the namespace we have created.

**Exercise**: evaluate this expression:
```python
def f(x):
    return x[1] > 0
print(19 if f([-2, -3, -4, -2]) else 18)
```

Go to new namespace:

  Inside `f`:
```python
x: [-2, -3, -4, -2] # create new name
```
Run the body:
```python
return x[1] > 0
```

Substitute
```python
return -3 > 0
```

Substitute
```python
return False
```

Go back to enclosing namespace:
```python
print(19 if f([-2, -3, -4, -2]) else 18)
```
and substitute:
```python
print(19 if False else 18)
```

Substitute
```python
print(18)
```

By the way, this strange inside-out if-else-statement is Python's way of writing the *ternary expression* which would be written `f() ? 19 : 18` in Java or C.

### Mentally simulating the interpreter

* The substitution model (like arithmetic)
* The environment model (namespaces and scope)
* Operator precedence (use parentheses when in doubt)
* Implicit state (e.g. in for-loops).


Mentally simulating the interpreter is a core skill in reading and writing code. It means pretending to "be" the interpreter, and knowing what it knows.